In [3]:
import networkx as nx
import EoN
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
from contact_tracing import *

import math
from constraint import ProbMinExposed, load_graph
import networkx as nx
import matplotlib.pyplot as plt
from collections import defaultdict
from ortools.linear_solver import pywraplp
from ortools.linear_solver.pywraplp import Constraint, Solver, Variable, Objective

from solve import *

from typing import Set, Dict
import pickle as pkl

import os
os.getcwd()

'/home/zach/Documents/prog/research/ContactTracing/ctrace'

In [4]:
G=grab_graph() #fetching the montgamery graph
n = len(G.nodes)

G1=load_graph("montgomery")
print(f"grab: {n} load: {len(G1.nodes)}")
I0 = [i for i in range(n) if random.random() > 0.999]

grab: 74555 load: 75457


In [ ]:
N = G.number_of_nodes()

# COSTS = np.random.randint(1, 20, size=N)
COSTS = np.ones(N)
print(f"COSTS: {COSTS}")
# Compute distances
dist_dict = nx.multi_source_dijkstra_path_length(G, I0)

# convert dict vertex -> distance
# to distance -> [vertex]
level_dists = defaultdict(set)
for (i, v) in dist_dict.items():
    level_dists[v].add(i)

# print(level_dists)

# Obtain V_1 and V_2

# Set of vertices distance 1 away from infected I
V_1: Set[int] = level_dists[1]

# Set of vertices distance 2 away from infected I
V_2: Set[int] = level_dists[2]

In [ ]:
p1 = defaultdict(lambda: 1)
q = defaultdict(lambda: defaultdict(lambda : 1))

prob = ProbMinExposed(G,I0,V_1,V_2,p1,q,10,COSTS)

In [ ]:
prob.solve_lp()
probabilities = np.array(prob.getVariables())

In [ ]:
iterated_round(prob, 11)

In [ ]:
"""STOP HERE"""

In [ ]:
len(V_1)

In [ ]:
V1 = set([])
for i in I0:
    for j in G.neighbors(i):
        if j not in I0:
            V1.add(j)


In [ ]:
len(Q['v'].unique())

In [ ]:
len(I0)

In [ ]:
len(V_2)

In [ ]:
P,Q = PQ(G,I0,runs = 20)

In [ ]:
for u in V1:
    for v in G.neighbors(u):
        if v in V2:
            print(u)
            print(v)
            #coeff = (q[u][v] * self.p1[u])
            #print(f"Coeff: {coeff}")
            #self.solver.Add(self.Y2[v] >= coeff * self.Y1[u])

In [ ]:
G.adj[1076]

In [ ]:
Q[Q.u == 1076]

In [ ]:
prob.getVariables()

In [ ]:
#test simplify

a = 1
b = 0

total_a = 0
total_b = 0

n=1000000

for i in range(n):
    (x,y,new_a,new_b) = simplify(a,b)
    
    if x==-1:
        total_a += new_a
        total_b += y
    elif y==-1:
        total_a += x
        total_b += new_b
    else:
        total_a += x
        total_b += y

print(str(total_a/n) + " " + str(a))
print(str(total_b/n) + " " + str(b))

In [ ]:
#test D
p = [.5,.5,.25,.75]

n=10000
total = np.zeros(4)

for i in range(n):
    a = D(p)
    if np.sum(a) != np.sum(p):
        print("oof")
    #print(p)
    total += a
    
total/=n
print(total)

In [ ]:
#test D_prime
p = [.5,.4,.25,.75]

n=5000
total = np.zeros(4)

for i in range(n):
    a = D(p)
    if (np.sum(a) != math.floor(np.sum(p))) & (np.sum(a) != math.ceil(np.sum(p))):
        print("oof")
    #print(p)
    total += a
    
total/=n
print(total)